In [313]:
# Virtual Environment
# All the units are in SI system
import numpy as np
# 

# Virtual Environment
# All the units are in SI system
import numpy as np
# 

def Matrial_on_the_road(Air_temp, Suraface_temp, Precipitation, wind_vel,initial_Snow, initial_water, initial_Salt, initial_ice, plowing):
    # General assumptions:
    Dataset_hour_inc = 1 # *3 had applied in the dataset
    f_t = 0.9 #times when the road is covered with moving vehicle
    g_t = 1 - f_t
    
    
    
    # convert Precipitation to snow or rain based on the air temp
    if Air_temp < .5:
        Snowfall_amount = Precipitation
        Rainfall_amount = 0
    else:
        Rainfall_amount = Precipitation
        Snowfall_amount = 0
    
    ###### Heat Ballance ##########################################################################################################
    # Q_csp : Flux of pavement heat
    V_wis = initial_ice + initial_Snow + Snowfall_amount # Volume of WIS layer (m3*m-2 = m) depth of material on the road
    V_ps = 50e-3 # thikness of pavement surface, generally 25 to 55 mm, here we assume 50mm
    Lambda_wis = 0.8 # thermal conductivity of WIS layer. Assume that is compacted snow 
    Lambda_P = 1.5 # Thermal conductivity of pavemetn. 0.8 to 2
    T_wis = (Air_temp + Suraface_temp)/2 #Temp of wis layer: assumption 
    Q_csp = 1/((V_wis/(2*Lambda_wis))+(V_ps/(2*Lambda_P)))*(Suraface_temp - T_wis)
    
     # Q_rn : Flux of net radient heat
    q_rld = 20 #Since we are modeling the winter stroms, the sky radiation shouldn't be high 0-200
    q_rlu = 0.97 * 5.64e-8 * (T_wis + 273.15)**4
    q_rsd = 20 #Since we are modeling the winter stroms, the sky radiation shouldn't be high 0-300
    q_rsu = 0.3 * q_rsd
    Q_rn = f_t*q_rld + q_rlu + f_t*q_rsd - q_rsu
    
    # Q_sn Flux of net radiant heat
    q_sa = (10e4 * wind_vel**0.7 +2.2)*(T_wis - Air_temp)#snesable heat flux due to wind
    q_sf = (4.184* (initial_water + Rainfall_amount) + 2.108 * (initial_ice  + initial_Snow + Snowfall_amount))# Sensable heat flux due to rainfall and snowfall
    q_sr = 0 #Sensible heat flux of drainage due to road gradient assume = 0
    q_sv = 4.184 * T_wis * 1000 * 0.5 * 0.005/3600 * 0.15#sensible heat flux of water dispersion due to passing vehicle
    Q_sn = q_sa + f_t*q_sf + q_sr + g_t * q_sv
    
    # Q_ln flux of net latent heat
    # m_wi * L_wi is not considered here, we will change the q_net relation with M_wi later
    m_il = 0 # sublimation flux
    L_i = 2838 #kJkg-1 latent heat of sublimation
    m_wl = 0 # flux of evaporation and condensation
    L_w = 2260 #kJkg-1 latent heat of evaporation and condensation
    m_sl = 3.34e-5 # dissolving flux (0 - 6.67e-5) if salting
    L_s = -66.4 # latent heat of dissolution of salt 
    Q_ln = m_il*L_i + m_wl*L_w + m_sl*L_s
    
    # Q_vn flux of net vehicle heat
    Q_vn = 100 #assumption according to Fujimoto and other citations
    
    # Q_net
    Q_net = Q_csp + Q_rn + Q_sn + Q_ln + Q_vn
    #################################################################################################################################
    #*******************************************************************************************************************************#
    
    ## Material creation $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    # Ice 
    Ice_final = initial_ice 
    
    # Snowfall 
    m_snowf = Snowfall_amount * Dataset_hour_inc * 100 # 100 kg/m3
    Snow_creat = m_snowf * f_t
    Snow_final = Snow_creat + initial_Snow
    # plowing  $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    if plowing == 1:
        Snow_final *= 0.1
        
    # or rainfall
    m_wf = Rainfall_amount * Dataset_hour_inc * 1000 # 1000 kg/m3
    Rain_creat = m_wf * f_t
    Water_final = Rain_creat + initial_water
    # before mesuring the water to ice, we have to consider the water dispersion
    # Water dispersion
    # Assume we have good drainage system
    Water_final = Water_final * 0.95
    
    #Salt
    # Salt_final = initial_Salt * 0.5# Maybe need conversion
    
    ### Material Conversion $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    ## Snow to ice
    Snow_To_Ice_rate = 0.1 # Assumption: conversion applied to initial snow 
    Ice_final += Snow_To_Ice_rate * initial_Snow
    
    ## water to ice or vise versa *
    L_wi = 334 #kJkg-1 latent heat of melting and freezing
    Water_ice_conv_mass = abs(Q_net/(L_wi*999))
    # We will assume that the water freezing point is just a function of salt concentration
    T_frezzing = -0.025 * initial_Salt - 0.5
    # print("TF",T_frezzing)
    
    if T_wis > T_frezzing:
        STATUS = "Melting"
        # print("###############################Melting")  # Melting
        stat_sign = 1
        Ice_final = Ice_final - min(Water_ice_conv_mass, Ice_final)  
        Water_final = Water_final + min(Water_ice_conv_mass, Ice_final) 
    else:
        STATUS = "Freezing"
        # print("###############################Freezing")  # Freezing
        Ice_final = Ice_final + min(Water_ice_conv_mass, Water_final)  
        Water_final = Water_final - min(Water_ice_conv_mass, Water_final) 
    

    # Snow to water
    if STATUS == "Melting":
        Water_final = Water_final + stat_sign * Water_ice_conv_mass
        Water_final = Water_final * 0.05 #disperssed
        Snow_final = Snow_final - stat_sign * Water_ice_conv_mass
        # print ("Water_ice_conv_mass", Water_ice_conv_mass)
    ## Material dispersion $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    # Salt dispersion: depands on STATUS
    if STATUS == "Melting":
        Salt_disp_rate = 0.3
    if STATUS == "Freezing":
        Salt_disp_rate = 0.9
    Salt_final = initial_Salt * Salt_disp_rate
    
    
    # Check each variable and set to zero if negative
    Water_final = max(Water_final, 0)
    Salt_final = max(Salt_final, 0)
    Ice_final = max(Ice_final, 0)
    Snow_final = max(Snow_final, 0)
    
    
    # print ("Water_final, Salt_final, Ice_final, Snow_final", Water_final, Salt_final, Ice_final, Snow_final)
    return Water_final, Salt_final, Ice_final, Snow_final
print ("done")


done


In [314]:
import numpy as np
import pandas as pd
from collections import deque
import random
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

class DQNAgent:
    def __init__(self, state_size, action_size, model_path=None):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95  # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.learning_rate = 0.1
        if model_path:
            self.model = load_model(model_path)
            print("Model loaded from", model_path)
        else:
            self.model = self._build_model()
            print("New model initialized")

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()
        model.add(Dense(24, input_dim=total_state_size, activation='relu'))
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(learning_rate=self.learning_rate))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            # Biased action selection
            actions_probability = [0.25, 0.25, 0.25, 0.25]  # Example: 70% do nothing, 10% for each other action
            return np.random.choice(np.arange(self.action_size), p=actions_probability)
        else:
            act_values = self.model.predict(state)
            return np.argmax(act_values[0])
    
    def predict_action(self, state):
        """Selects the action with the highest Q-value for the given state."""
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state)[0]))
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            
            
print ("done")

done


In [315]:

import numpy as np

class RoadEnv:
    def __init__(self, dataset):
        self.dataset = dataset
        self.current_state = None
        self.action_costs = {0: 0, 1: 1, 2: 6, 3: 5}
        self.reset()
    def simulate_no_intervention(self, window_data):
        no_intervention_results = []
        initial_conditions = {'initial_Snow': 0, 'initial_water': 0, 'initial_Salt': 0, 'initial_ice': 0, 'plowing':0}
        for index, row in window_data.iterrows():
            # Simulate step without interventions
            Air_temp = row['Air TemperatureC']
            Surface_temp = row['Surface TemperatureC']
            Precipitation = row['Precipitation Intensitym/3h'] 
            wind_vel = row['Wind Speed (act)m/s']
            water, salt, ice, snow = Matrial_on_the_road(Air_temp, Surface_temp, Precipitation, wind_vel, **initial_conditions)
            # print ("water, salt, ice, snow", water, salt, ice, snow)
            # Update initial conditions for the next step
            initial_conditions.update({'initial_Snow': snow, 'initial_ice': ice, 'initial_water': water, 'initial_Salt': salt, 'plowing': 0})
            
            # Append results for no intervention forecast
            no_intervention_results.append((water, salt, ice, snow))
            # print("no_intervention_results",no_intervention_results)
        return no_intervention_results
    # def reset(self):
    #     # Reset the environment to the initial state of a new episode
    #     self.current_state = self.dataset.iloc[0].copy()
    #     self.plowing_count = 0
    #     self.salting_count = 0
    #     
    #     return self._get_state()
    def reset(self, episode_index=0):
        # Calculate the start index of the episode in the dataset
        start_index = episode_index * 10  # Assuming each episode has 10 states
        
        # Check if the calculated start index is within the bounds of the dataset
        if start_index >= 0 and start_index < len(self.dataset):
            self.current_state = self.dataset.iloc[start_index].copy()
        else:
            # Fallback to the first row if the calculated index is out of bounds
            self.current_state = self.dataset.iloc[0].copy()
            print(f"Warning: Episode index {episode_index} out of bounds, falling back to initial state.")
    
        # Reset the action counts for the new episode
        self.plowing_count = 0
        self.salting_count = 0
        
        # Reset environment to the start of a new episode, with modifications to include no intervention forecast
        window_id = unique_window_ids[episode_index]
        episode_data = self.dataset[self.dataset['Window_ID'] == window_id]
        self.no_intervention_forecast = self.simulate_no_intervention(episode_data)
        # print ("self.no_intervention_forecast", self.no_intervention_forecast)
        # Return the initial state of the new episode
        return self._get_state()

    def step(self, action):
        reward = 0
        # Check if the action is allowed based on the resource limitations
        if action == 1 and self.plowing_count >= 2:  # P`lowing limit reached
            reward = -1000  # Force 'no action'
        elif action == 3 and self.salting_count >= 1:  # Salting limit reached
            reward = -1000  # Force 'no action'
        elif action == 2:  # Plowing and salting
            if self.plowing_count >= 2 or self.salting_count >= 1:
                reward = -1000  # If either limit is reached, force 'no action'

    
        # Update action counters
        if action == 1:  # Plowing
            self.plowing_count += 1
        elif action == 3:  # Salting
            self.salting_count += 1
        elif action == 2:  # Plowing and salting
            self.plowing_count += 1
            self.salting_count += 1
        # Apply the chosen action to the environment and update the state
        # print ("action", action)
        water, salt, ice, snow = self._apply_action(action)
        # print (" water, salt, ice, snow", water, salt, ice, snow)
        # Update the current state with the new values
        self.current_state['initial_Snow'] = snow
        self.current_state['initial_water'] = water
        self.current_state['initial_Salt'] = salt
        self.current_state['initial_ice'] = ice

        # Calculate the reward
        reward += self._calculate_reward(action, ice, snow)
        # print ("reward",reward)
        # Move to the next row in the dataset, simulating time progression
        
        # Dynamically determine if it's the last state in the episode
        current_episode_index = self.dataset.index.get_loc(self.current_state.name)
        episode_start_index = current_episode_index - (current_episode_index % 10)  # Assuming episodes start at indices 0, 10, 20, ...
        next_index = current_episode_index + 1
        is_last_state_in_episode = (next_index % 10 == 0) or (next_index >= len(self.dataset))
        
        if is_last_state_in_episode:
            done = True
        else:
            self.current_state = self.dataset.iloc[next_index].copy()
            done = False
    
        return self._get_state(), reward, done

    def _get_state(self):
        # Extract the current state from the dataset row
        current_state_values = [self.current_state[col] for col in ['Air TemperatureC', 'Surface TemperatureC', 
                                                                     'Wind Speed (act)m/s', 'Precipitation Intensitym/3h', 
                                                                     'initial_Snow', 'initial_water', 'initial_Salt', 'initial_ice']]
        
        # Prepare no intervention forecast data for the entire episode
        # Assuming no_intervention_forecast is a list of tuples for the entire episode at this point
        no_intervention_forecast_flat = [value for forecast in self.no_intervention_forecast for value in forecast]
    
        # Combine the current state values with the no intervention forecast
        # Here, we ensure the state_with_forecast includes the no intervention data for the entire episode
        state_with_forecast = np.array(current_state_values + no_intervention_forecast_flat)
        # print ("state_with_forecast", state_with_forecast)
        return state_with_forecast

    def _apply_action(self, action):
        # Apply the chosen action and update the road conditions
        # For simplicity, this function assumes the existence of a function named 'Matrial_on_the_road'
        # that calculates the final amounts of water, salt, ice, and snow

        air_temp = self.current_state['Air TemperatureC']
        surface_temp = self.current_state['Surface TemperatureC']
        precipitation = self.current_state['Precipitation Intensitym/3h']
        wind_vel = self.current_state['Wind Speed (act)m/s']
        initial_snow = self.current_state['initial_Snow']
        initial_water = self.current_state['initial_water']
        initial_salt = self.current_state['initial_Salt']
        initial_ice = self.current_state['initial_ice']
        plowing  = 0
        if action == 1 or action == 2:  # Plowing or Plowing and Salting
            plowing  = 1
            

        if action == 2 or action == 3:  # Salting or Plowing and Salting
            # Measuring salt amount based on the Ruled-Base model
                Salt_amount = -32.027 * surface_temp + 79.24
                if Salt_amount < 0: Salt_amount = 0
                if Salt_amount > 400: Salt_amount = 400  # here salt unit is lb/mile/lane
                initial_salt += Salt_amount  # Assuming a constant amount of salt is used

        water_final, salt_final, ice_final, snow_final = Matrial_on_the_road(
            air_temp, surface_temp, precipitation, wind_vel,
            initial_snow, initial_water, initial_salt, initial_ice , plowing
        )
        # print (f"action {action}, water_final {water_final}, salt_final {salt_final}, ice_final {ice_final}, snow_final {snow_final}" )
        return water_final, salt_final, ice_final, snow_final

    def _calculate_reward(self, action, ice, snow):
        # Calculate the reward based on the reduction of ice and snow, and the cost of the action
        cost = self.action_costs[action]
        # Assuming a penalty of 20 pizoo for each cm of ice or snow
        penalty = (ice + snow) * 2000 
        reward = -penalty - cost
        
        return reward

print ("done")

done


In [ ]:
#Training
import pandas as pd
import numpy as np


# Assuming the DQNAgent and RoadEnv classes are defined as in your previous code

# Load and preprocess dataset
file_path = 'Modified_DataSetV8_ConvertedUnits.csv'
dataset = pd.read_csv(file_path)
for col in ['initial_Snow', 'initial_water', 'initial_Salt', 'initial_ice']:
    if col not in dataset.columns:
        dataset[col] = 0

# Initialize environment and agent
env = RoadEnv(dataset)

action_size = 4
no_intervention_data_size = 10 * 4  # 10 future states * 4 variables per state
current_state_size = 8  # Current state variables
total_state_size = current_state_size + no_intervention_data_size

agent = DQNAgent(total_state_size, action_size)


# Split dataset into training and validation
unique_window_ids = dataset['Window_ID'].unique()
total_training_windows = len(unique_window_ids)
print(f"Total training window IDs: {total_training_windows}")

# Use only the first 100 episodes for training, excluding the last 10 for validation
train_window_ids = unique_window_ids[:4500]  # Adjusted to use only the first 100 episodes
validation_window_ids = unique_window_ids[-10:]  # Last 10 for validation

Current_training_windows = len(train_window_ids)
print(f"Current training window IDs: {Current_training_windows}")

# Training
batch_size = 8

def run_episode_simulation(agent, current_episode_data, state_size, episode_num):
    best_trial_reward = -np.inf
    best_trial_data = None

    # Initial simulation setup
    for trial in range(N_TRIALS):
        # print(f"Trial {trial}")
        trial_data = {'states': [], 'actions': [], 'rewards': [], 'next_states': [], 'dones': []}
        state = env.reset(episode_num)
        # print ("state",state)
        cumulative_reward = 0
        # print ("current_episode_data",current_episode_data)
        for index, row in current_episode_data.iterrows():
            state = np.reshape(state, [1, total_state_size])
            # print ("state", state)
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            # print ("next_state",next_state)
            next_state = np.reshape(next_state, [1, state_size])

            # Save trial data
            trial_data['states'].append(state)
            trial_data['actions'].append(action)
            trial_data['rewards'].append(reward)
            trial_data['next_states'].append(next_state)
            trial_data['dones'].append(done)
            
            state = next_state
            cumulative_reward += reward

            if cumulative_reward > best_trial_reward:
                best_trial_reward = cumulative_reward
                best_trial_data = trial_data
        # print ("best_trial_data", best_trial_data)
    return best_trial_data

N_TRIALS = 100  # Number of trials to run for each episode

for i, window_id in enumerate(train_window_ids):
    print(f"****************************************Processing Window ID {window_id} ({i+1}/{Current_training_windows})")
    current_episode_data = dataset[dataset['Window_ID'] == window_id]
    # print ("current_episode_data", current_episode_data)
    # Run episode simulation and get the best trial data
    # print ("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii", i)
    best_trial_data = run_episode_simulation(agent, current_episode_data, total_state_size, episode_num = i)
    # print ("best_trial_data")
    # print ("best_trial_data['states']", best_trial_data['states'])
    # print ("best_trial_data['actions']", best_trial_data['actions'])
    # print ("best_trial_data['rewards']", best_trial_data['rewards'])
    # print ("best_trial_data['next_states']",best_trial_data['next_states'])
    # print ("best_trial_data['dones']", best_trial_data['dones'])
    # Store the best trial's outcomes in memory
    for state, action, reward, next_state, done in zip(best_trial_data['states'], best_trial_data['actions'], best_trial_data['rewards'], best_trial_data['next_states'], best_trial_data['dones']):
        agent.remember(state, action, reward, next_state, done)
    print ("len(agent.memory)", len(agent.memory), "batch_size", batch_size)
    if len(agent.memory) > batch_size:
        agent.replay(batch_size)


# Save the trained model
agent.model.save('dqn_road_maintenance_model4500.h5')

print ("Training complete")



1/1 [==============================] - 0s 51ms/step


In [301]:
# Validation 11111111111111111111111111111111111
# Validation
print("Starting Validation...")
for window_id in validation_window_ids:
    current_episode_data = dataset[dataset['Window_ID'] == window_id]
    state = env.reset()  # Reset environment and action counters
    episode_actions = []
    episode_info = []
    cumulative_reward = 0

    for index, row in current_episode_data.iterrows():
        state = np.reshape(state, [1, total_state_size])
        
        # Use predict_action to exploit the learned policy
        action = agent.predict_action(state)
        
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, [1, total_state_size])
        cumulative_reward += reward

        episode_actions.append(action)
        episode_info.append({
            'state': state.flatten().tolist(),
            'action': action,
            'reward': reward,
            'next_state': next_state.flatten().tolist(),
            'cumulative_reward': cumulative_reward,
            'done': done
        })

        state = next_state
        if done:
            break

    print(f"\nEpisode for Window ID: {window_id}")
    print("Actions taken:", episode_actions)
    print("Cumulative Reward:", cumulative_reward)
    for info in episode_info:
        print(info)



Starting Validation...
1/1 [==============================] - 0s 46ms/step

Episode for Window ID: Tipler_window_990
Actions taken: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Cumulative Reward: -3931.2
{'state': [-11.58, -11.06, 2.27, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.34290000000000004, 0.0, 0.0, 0.03429000000000001, 0.8001, 0.0, 0.0, 0.11430000000000001, 1.3256999999999999, 0.0, 0.0, 0.24687, 1.6911, 0.0, 0.0, 0.41598, 1.9197, 0.0, 0.0, 0.60795, 1.9656], 'action': 0, 'reward': 0.0, 'next_state': [-9.65, -10.8, 2.92, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.34290000000000004, 0.0, 0.0, 0.03429000000000001, 0.8001, 0.0, 0.0, 0.11430000000000001, 1.3256999999999999, 0.0, 0.0, 0.24687, 1.6911, 0.0, 0.0, 0.41598, 1.9197, 0.0, 0.0, 0.60795, 1.9656], 'cumulative_reward': 0.0, 'done': False}
{'state': [-9.65, -10.8, 2.92, 0.0, 0.0, 0.0, 0

In [ ]:
#Training 5000-10000
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque
import random

# Assuming the DQNAgent and RoadEnv classes are defined as in your previous code


action_size = 4
no_intervention_data_size = 10 * 4  # 10 future states * 4 variables per state
current_state_size = 8  # Current state variables
total_state_size = current_state_size + no_intervention_data_size

agent = DQNAgent(total_state_size, action_size, model_path='dqn_road_maintenance_model5000.h5')


# Split dataset into training and validation
unique_window_ids = dataset['Window_ID'].unique()
total_training_windows = len(unique_window_ids)
print(f"Total training window IDs: {total_training_windows}")

# Use only the first 100 episodes for training, excluding the last 10 for validation
train_window_ids = unique_window_ids[5000:10000]  # Adjusted to use only the first 100 episodes
validation_window_ids = unique_window_ids[-10:]  # Last 10 for validation

Current_training_windows = len(train_window_ids)
print(f"Current training window IDs: {Current_training_windows}")

# Training
batch_size = 8

def run_episode_simulation(agent, current_episode_data, total_state_size, episode_num):
    best_trial_reward = -np.inf
    best_trial_data = None

    # Initial simulation setup
    for trial in range(N_TRIALS):
        # print(f"Trial {trial}")
        trial_data = {'states': [], 'actions': [], 'rewards': [], 'next_states': [], 'dones': []}
        state = env.reset(episode_num)
        # print ("state",state)
        cumulative_reward = 0
        # print ("current_episode_data",current_episode_data)
        for index, row in current_episode_data.iterrows():
            state = np.reshape(state, [1, total_state_size])
            # print ("state", state)
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            # print ("next_state",next_state)
            next_state = np.reshape(next_state, [1, total_state_size])

            # Save trial data
            trial_data['states'].append(state)
            trial_data['actions'].append(action)
            trial_data['rewards'].append(reward)
            trial_data['next_states'].append(next_state)
            trial_data['dones'].append(done)
            
            state = next_state
            cumulative_reward += reward

            if cumulative_reward > best_trial_reward:
                best_trial_reward = cumulative_reward
                best_trial_data = trial_data
        # print ("best_trial_data", best_trial_data)
    return best_trial_data

N_TRIALS = 5  # Number of trials to run for each episode

for i, window_id in enumerate(train_window_ids):
    print(f"****************************************Processing Window ID {window_id} ({i+1}/{Current_training_windows})")
    current_episode_data = dataset[dataset['Window_ID'] == window_id]
    # print ("current_episode_data", current_episode_data)
    # Run episode simulation and get the best trial data
    # print ("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii", i)
    best_trial_data = run_episode_simulation(agent, current_episode_data, total_state_size, episode_num = i)
    # print ("best_trial_data")
    # print ("best_trial_data['states']", best_trial_data['states'])
    # print ("best_trial_data['actions']", best_trial_data['actions'])
    # print ("best_trial_data['rewards']", best_trial_data['rewards'])
    # print ("best_trial_data['next_states']",best_trial_data['next_states'])
    # print ("best_trial_data['dones']", best_trial_data['dones'])
    # Store the best trial's outcomes in memory
    for state, action, reward, next_state, done in zip(best_trial_data['states'], best_trial_data['actions'], best_trial_data['rewards'], best_trial_data['next_states'], best_trial_data['dones']):
        agent.remember(state, action, reward, next_state, done)

    if len(agent.memory) > batch_size:
        agent.replay(batch_size)


# Save the trained model
agent.model.save('dqn_road_maintenance_model10000.h5')

print ("Training complete")


In [ ]:
# Validation 222222222222222222222222222222222222222222
print("Starting Validation 2")
for window_id in validation_window_ids:
    current_episode_data = dataset[dataset['Window_ID'] == window_id]
    state = env.reset()  # Ensure this resets both the state and action counters
    episode_actions = []
    episode_info = []
    cumulative_reward = 0
    for index, row in current_episode_data.iterrows():
        state = np.reshape(state, [1, total_state_size])
        action = agent.act(state)
        
        # Get the next state, reward, and whether the episode is done, considering action restrictions
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, [1, total_state_size])
        cumulative_reward += reward
        # Store action and state info
        episode_actions.append(action)
        episode_info.append({
            'state': state.flatten().tolist(),
            'action': action,
            'reward': reward,
            'next_state': next_state.flatten().tolist(),
            'cumulative_reward': cumulative_reward,
            'done': done
        })

        state = next_state
        if done:
            break

    # Print episode information
    print(f"\nEpisode for Window ID: {window_id}")
    print("Actions taken:", episode_actions)
    for info in episode_info:
        print(info)


In [ ]:
#Training 5000-10000
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque
import random

# Assuming the DQNAgent and RoadEnv classes are defined as in your previous code


action_size = 4
no_intervention_data_size = 10 * 4  # 10 future states * 4 variables per state
current_state_size = 8  # Current state variables
total_state_size = current_state_size + no_intervention_data_size

agent = DQNAgent(total_state_size, action_size, model_path='dqn_road_maintenance_model10000.h5')


# Split dataset into training and validation
unique_window_ids = dataset['Window_ID'].unique()
total_training_windows = len(unique_window_ids)
print(f"Total training window IDs: {total_training_windows}")

# Use only the first 100 episodes for training, excluding the last 10 for validation
train_window_ids = unique_window_ids[10000:20000]  # Adjusted to use only the first 100 episodes
validation_window_ids = unique_window_ids[-10:]  # Last 10 for validation

Current_training_windows = len(train_window_ids)
print(f"Current training window IDs: {Current_training_windows}")

# Training
batch_size = 8
N_TRIALS = 100

def run_episode_simulation(agent, current_episode_data, total_state_size, episode_num):
    best_trial_reward = -np.inf
    best_trial_data = None

    # Initial simulation setup
    for trial in range(N_TRIALS):
        # print(f"Trial {trial}")
        trial_data = {'states': [], 'actions': [], 'rewards': [], 'next_states': [], 'dones': []}
        state = env.reset(episode_num)
        # print ("state",state)
        cumulative_reward = 0
        # print ("current_episode_data",current_episode_data)
        for index, row in current_episode_data.iterrows():
            state = np.reshape(state, [1, total_state_size])
            # print ("state", state)
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            # print ("next_state",next_state)
            next_state = np.reshape(next_state, [1, total_state_size])

            # Save trial data
            trial_data['states'].append(state)
            trial_data['actions'].append(action)
            trial_data['rewards'].append(reward)
            trial_data['next_states'].append(next_state)
            trial_data['dones'].append(done)
            
            state = next_state
            cumulative_reward += reward

            if cumulative_reward > best_trial_reward:
                best_trial_reward = cumulative_reward
                best_trial_data = trial_data
        # print ("best_trial_data", best_trial_data)
    return best_trial_data
# Define constants
EPISODES_BEFORE_REPLAY = 5  # Number of episodes before replaying experiences
REPLAY_BATCH_SIZE = 8  # Batch size for replay
  # Number of trials to run for each episode

# Training loop
for i, window_id in enumerate(train_window_ids):
    print(f"****************************************Processing Window ID {window_id} ({i+1}/{Current_training_windows})")
    current_episode_data = dataset[dataset['Window_ID'] == window_id]

    # Run episode simulation and get the best trial data
    best_trial_data = run_episode_simulation(agent, current_episode_data, total_state_size, episode_num=i)
    
    # Store the best trial's outcomes in memory
    for state, action, reward, next_state, done in zip(best_trial_data['states'], best_trial_data['actions'], best_trial_data['rewards'], best_trial_data['next_states'], best_trial_data['dones']):
        agent.remember(state, action, reward, next_state, done)

    # Check if it's time for replay
    if (i + 1) % EPISODES_BEFORE_REPLAY == 0:
        # Replay experiences
        for _ in range(REPLAY_BATCH_SIZE):
            if len(agent.memory) > batch_size:
                agent.replay(REPLAY_BATCH_SIZE)

# Save the trained model
agent.model.save('dqn_road_maintenance_model20000.h5')

print("Training complete")


In [305]:
# Validation 33333333333333333333333333333333333333
print("Starting Validation 3")
for window_id in validation_window_ids:
    current_episode_data = dataset[dataset['Window_ID'] == window_id]
    state = env.reset()  # Ensure this resets both the state and action counters
    episode_actions = []
    episode_info = []
    cumulative_reward = 0
    for index, row in current_episode_data.iterrows():
        state = np.reshape(state, [1, total_state_size])
        action = agent.act(state)
        
        # Get the next state, reward, and whether the episode is done, considering action restrictions
        next_state, reward, done = env.step(action)
        next_state = np.reshape(next_state, [1, total_state_size])
        cumulative_reward += reward
        # Store action and state info
        episode_actions.append(action)
        episode_info.append({
            'state': state.flatten().tolist(),
            'action': action,
            'reward': reward,
            'next_state': next_state.flatten().tolist(),
            'cumulative_reward': cumulative_reward,
            'done': done
        })

        state = next_state
        if done:
            break

    # Print episode information
    print(f"\nEpisode for Window ID: {window_id}")
    print("Actions taken:", episode_actions)
    for info in episode_info:
        print(info)


Starting Validation 3
1/1 [==============================] - 0s 46ms/step

Episode for Window ID: Tipler_window_990
Actions taken: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
{'state': [-11.58, -11.06, 2.27, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.34290000000000004, 0.0, 0.0, 0.03429000000000001, 0.8001, 0.0, 0.0, 0.11430000000000001, 1.3256999999999999, 0.0, 0.0, 0.24687, 1.6911, 0.0, 0.0, 0.41598, 1.9197, 0.0, 0.0, 0.60795, 1.9656], 'action': 0, 'reward': 0.0, 'next_state': [-9.65, -10.8, 2.92, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.34290000000000004, 0.0, 0.0, 0.03429000000000001, 0.8001, 0.0, 0.0, 0.11430000000000001, 1.3256999999999999, 0.0, 0.0, 0.24687, 1.6911, 0.0, 0.0, 0.41598, 1.9197, 0.0, 0.0, 0.60795, 1.9656], 'cumulative_reward': 0.0, 'done': False}
{'state': [-9.65, -10.8, 2.92, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,

In [ ]:
#Training 5000-10000
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from collections import deque
import random

# Assuming the DQNAgent and RoadEnv classes are defined as in your previous code


action_size = 4
no_intervention_data_size = 10 * 4  # 10 future states * 4 variables per state
current_state_size = 8  # Current state variables
total_state_size = current_state_size + no_intervention_data_size

agent = DQNAgent(total_state_size, action_size, model_path='dqn_road_maintenance_model20000.h5')


# Split dataset into training and validation
unique_window_ids = dataset['Window_ID'].unique()
total_training_windows = len(unique_window_ids)
print(f"Total training window IDs: {total_training_windows}")

# Use only the first 100 episodes for training, excluding the last 10 for validation
train_window_ids = unique_window_ids[20000:30000]  # Adjusted to use only the first 100 episodes
validation_window_ids = unique_window_ids[-10:]  # Last 10 for validation

Current_training_windows = len(train_window_ids)
print(f"Current training window IDs: {Current_training_windows}")

# Training
batch_size = 8

def run_episode_simulation(agent, current_episode_data, total_state_size, episode_num):
    best_trial_reward = -np.inf
    best_trial_data = None

    # Initial simulation setup
    for trial in range(N_TRIALS):
        # print(f"Trial {trial}")
        trial_data = {'states': [], 'actions': [], 'rewards': [], 'next_states': [], 'dones': []}
        state = env.reset(episode_num)
        # print ("state",state)
        cumulative_reward = 0
        # print ("current_episode_data",current_episode_data)
        for index, row in current_episode_data.iterrows():
            state = np.reshape(state, [1, total_state_size])
            # print ("state", state)
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            # print ("next_state",next_state)
            next_state = np.reshape(next_state, [1, total_state_size])

            # Save trial data
            trial_data['states'].append(state)
            trial_data['actions'].append(action)
            trial_data['rewards'].append(reward)
            trial_data['next_states'].append(next_state)
            trial_data['dones'].append(done)
            
            state = next_state
            cumulative_reward += reward

            if cumulative_reward > best_trial_reward:
                best_trial_reward = cumulative_reward
                best_trial_data = trial_data
        # print ("best_trial_data", best_trial_data)
    return best_trial_data

N_TRIALS = 30  # Number of trials to run for each episode

for i, window_id in enumerate(train_window_ids):
    print(f"****************************************Processing Window ID {window_id} ({i+1}/{Current_training_windows})")
    current_episode_data = dataset[dataset['Window_ID'] == window_id]
    # print ("current_episode_data", current_episode_data)
    # Run episode simulation and get the best trial data
    # print ("iiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii", i)
    best_trial_data = run_episode_simulation(agent, current_episode_data, total_state_size, episode_num = i)
    # print ("best_trial_data")
    # print ("best_trial_data['states']", best_trial_data['states'])
    # print ("best_trial_data['actions']", best_trial_data['actions'])
    # print ("best_trial_data['rewards']", best_trial_data['rewards'])
    # print ("best_trial_data['next_states']",best_trial_data['next_states'])
    # print ("best_trial_data['dones']", best_trial_data['dones'])
    # Store the best trial's outcomes in memory
    for state, action, reward, next_state, done in zip(best_trial_data['states'], best_trial_data['actions'], best_trial_data['rewards'], best_trial_data['next_states'], best_trial_data['dones']):
        agent.remember(state, action, reward, next_state, done)

    if len(agent.memory) > batch_size:
        agent.replay(batch_size)


# Save the trained model
agent.model.save('dqn_road_maintenance_model30000.h5')

print ("Training complete")
